## Protocol to generate initial positions for PMF

In [1]:
import numpy as np
import MDAnalysis as md
from MDAnalysis.analysis import align
import subprocess
import networkx
import os
from MDAnalysis.analysis.leaflet import LeafletFinder

In [2]:
## Define paths to forcefield files, mdp files, insane script and an equilibrated water box
ff_loc  = 'ff_itp/'
mdp_loc = 'mdps/'
insane_loc = 'common_files'
water_loc = 'common_files'

IL_name = 'MC3H'

In [3]:
pin_cores = '-pin on -pinstride 1 -pinoffset 30'

### Functions are hidden here
===============================================================================================

In [4]:
def build_system(comp1, comp2, hydration, forcefields, insane_loc):
    
    # ### Generate Comp1 membrane
    subprocess.call(f"python  {insane_loc}/python3_insane_M3_lipids.py  {comp1} -pbc cubic -o comp1_mem.gro -p topol_comp1.top", shell = True)
    
    ### Generate Comp2 membrane
    subprocess.call(f"python  {insane_loc}/python3_insane_M3_lipids.py {comp2} -pbc cubic -o comp2_mem.gro -p topol_comp2.top", shell = True)
    
        
    ##################################################
    ### Equilibrate both membranes separately
    ###################################################
    
    ############################# Comp1 membrane ##############
    #### Renumber and generate box
    subprocess.call('gmx editconf -f comp1_mem.gro -o comp1_mem.gro -resnr 1'.format(mdp_loc), shell=True)

    ############# Create topology file
    with open('topol_comp1.top') as topinput:
        top = topinput.readlines()

    top[0]= forcefields
    top[7]= ';membrane1   number\n'

    ## write the file
    with open("topol_comp1_W.top", "w") as f:
        top = "".join(top)
        f.write(top)

    ##### Solvate system
    subprocess.call('gmx solvate -cp comp1_mem.gro -cs {}/water.gro -p topol_comp1_W.top -o comp1_mem_W.gro -radius 0.21'.format(water_loc)
                   , shell=True)

    ##### Neutralize charge
    tpr = 'ions.tpr'
    out = 'comp1_mem_W_ion.gro'
    top = 'topol_comp1_W.top'
    !touch empty.mdp

    subprocess.call('gmx grompp -f empty.mdp -c comp1_mem_W.gro -r comp1_mem_W.gro -p topol_comp1_W.top -o ions.tpr -renum -maxwarn 1', shell=True)

    p = subprocess.Popen(['gmx', 'genion', '-s', tpr, '-o', out, '-p', top, '-neutral'], stdin=subprocess.PIPE)
    p.communicate(b'W\n')
    p.wait()

    #### Water energy minimisation 
    subprocess.call('gmx grompp -f {}em_2.mdp -c comp1_mem_W_ion.gro -r comp1_mem_W_ion.gro -p topol_comp1_W.top -o em_comp1.tpr'.format(mdp_loc), shell=True)
    subprocess.call(f'gmx mdrun -v -deffnm em_comp1 -nt 10 {pin_cores}', shell=True)

    ## Remove trash
    !rm *#

    ###Create new index file
    gro = 'comp1_mem_W_ion.gro'
    u = md.Universe(gro)

    Lipids = u.select_atoms('not name W')
    Water = u.select_atoms('name W')

    Lipids.write("index_comp1_mem.ndx", mode="w", name= 'Solute')
    Water.write("index_comp1_mem.ndx", mode="a", name= 'Solvent')

    u.atoms.write("index_comp1_mem.ndx", mode="a", name= 'System')

    #### Equilibration 
    subprocess.call('gmx grompp -f {}eq_3_20ns.mdp -c em_comp1.gro -r em_comp1.gro -p topol_comp1_W.top -n index_comp1_mem.ndx -o eq1_comp1.tpr -maxwarn 1'.format(mdp_loc), shell=True)
    subprocess.call(f'gmx mdrun -v -deffnm eq1_comp1 -nt 10 {pin_cores}', shell=True)

    ## Remove trash
    !rm *#
    
    
    
    ############################### Comp2 membrane ##############
    #### Renumber and generate box
    subprocess.call('gmx editconf -f comp2_mem.gro -o comp2_mem.gro -resnr 1'.format(mdp_loc), shell=True)


    ############# Create topology file
    with open('topol_comp2.top') as topinput:
        top = topinput.readlines()

    top[0]= forcefields
    top[7]= ';membrane2\n'

    ## write the file
    with open("topol_comp2_W.top", "w") as f:
        top = "".join(top)
        f.write(top)


    ##### Solvate system
    subprocess.call('gmx solvate -cp comp2_mem.gro -cs {}/water.gro -p topol_comp2_W.top -o comp2_mem_W.gro -radius 0.21'.format(water_loc)
                   , shell=True)


    ##### Neutralize charge
    tpr = 'ions.tpr'
    out = 'comp2_mem_W_ion.gro'
    top = 'topol_comp2_W.top'
    !touch empty.mdp

    subprocess.call('gmx grompp -f empty.mdp -c comp2_mem_W.gro -r comp2_mem_W.gro -p topol_comp2_W.top -o ions.tpr -renum -maxwarn 1', shell=True)

    p = subprocess.Popen(['gmx', 'genion', '-s', tpr, '-o', out, '-p', top, '-neutral'], stdin=subprocess.PIPE)
    p.communicate(b'W\n')
    p.wait()


    #### Water energy minimisation 
    subprocess.call('gmx grompp -f {}em_2.mdp -c comp2_mem_W_ion.gro -r comp2_mem_W_ion.gro -p topol_comp2_W.top -o em_comp2.tpr'.format(mdp_loc), shell=True)
    subprocess.call(f'gmx mdrun -v -deffnm em_comp2 -nt 10 {pin_cores}', shell=True)

    ## Remove trash
    !rm *#


    ###Create new index file
    gro = 'comp2_mem_W_ion.gro'
    u = md.Universe(gro)

    Lipids = u.select_atoms('not name W')
    Water = u.select_atoms('name W')

    Lipids.write("index_comp2_mem.ndx", mode="w", name= 'Solute')
    Water.write("index_comp2_mem.ndx", mode="a", name= 'solvent')

    u.atoms.write("index_comp2_mem.ndx", mode="a", name= 'System')


    #### Equilibration 
    subprocess.call('gmx grompp -f {}eq_3_20ns.mdp -c em_comp2.gro -r em_comp2.gro -p topol_comp2_W.top -n index_comp2_mem.ndx -o eq1_comp2_in.tpr -maxwarn 1'.format(mdp_loc), shell=True)
    subprocess.call(f'gmx mdrun -v -deffnm eq1_comp2_in -nt 10 {pin_cores}', shell=True)

    
    #######################################
    ###### Define hydration level
    #######################################
    
    subprocess.call(f'gmx editconf -f eq1_comp2_in.gro -o eq1_comp2.gro -resnr 1 -box 10 10 {hydration}'.format(mdp_loc), shell=True)
    
    #######################################
    ###### Join both membranes  
    #######################################
    
    comp1_mem = md.Universe('eq1_comp1.gro')
    comp2_mem = md.Universe('eq1_comp2.gro')

    sys = md.Merge(comp1_mem.select_atoms('not name W NA CL'), comp2_mem.select_atoms('not name W NA CL'))

    ### write both membranes in one file 
    sys.atoms.write('stacked_membrane.gro')
    
    #### Renumber and generate box
    subprocess.call('gmx editconf -f stacked_membrane.gro -o stacked_membrane_renum.gro -resnr 1 -box 10 10 18'.format(mdp_loc), shell=True)
    
    ## Remove trash
    !rm *#    
    
    return

In [5]:
def area_per_lipid(gro, xtc):#, lipids):
    #### Calculate area per lipid 
    u = md.Universe(gro,xtc)
    lipids = u.select_atoms('name PO4 NP N1')

    ## Determine area per lipid
    areas = []
    times = []

    for ts in u.trajectory[:]:
        APL = (u.dimensions[0]*u.dimensions[1])/(len(lipids)/2)
        areas.append(APL)
        #times.append(ts.time)

    ## Calculate average and standard deviation    
    avg = np.mean(areas)
    std = np.std(areas)
    return(avg,std,lipids)

In [6]:
def minimize_equilibrate():
    #### Water energy minimisation 
    subprocess.call(f'gmx grompp -f {mdp_loc}em_2.mdp -c mem_solvent_ions.gro -r restraint_reference.gro -p system_W.top -o em2.tpr', shell=True)
    subprocess.call(f'gmx mdrun -v -deffnm em2 -nt 10 {pin_cores}', shell=True)

    ## Remove trash
    !rm *#
    
    ###Create new index file
    gro = 'em2.gro'
    u = md.Universe(gro)

    Lipids = u.select_atoms('not name W NA CL')
    Water = u.select_atoms('name W NA CL')

    Lipids.write("index_eq.ndx", mode="w", name= 'Solute')
    Water.write("index_eq.ndx", mode="a", name= 'solvent')

    u.atoms.write("index_eq.ndx", mode="a", name= 'System')
    
    #### Equilibration 
    subprocess.call(f'gmx grompp -f {mdp_loc}eq_3_20ns.mdp -c em2.gro -n index_eq.ndx -r restraint_reference.gro -p system_W.top -o eq1.tpr -maxwarn 1', shell=True)
    subprocess.call(f'gmx mdrun -v -deffnm eq1 -nt 10 {pin_cores}', shell=True)


    ## Remove trash
    !rm *#
    
    return

In [7]:
def neutralize(neg_lipids, pos_lipids):          
    ##########################################
    ############## Neutralize lipid charge
    ##########################################

    ######## Find lower leaflet of upper membrane
    u_UpperMembrane = md.Universe('eq1_comp2.gro')
    L_up = LeafletFinder(u_UpperMembrane, 'name PO4 NP', cutoff = 20)
    UpperMembrane_leaflet0 = L_up.groups(0)  ### upper
    UpperMembrane_leaflet1 = L_up.groups(1)  ### lower
    
    ######## Find upper leaflet of lower membrane
    u_LowerMembrane = md.Universe('eq1_comp1.gro')
    L_low = LeafletFinder(u_LowerMembrane, 'name PO4 NP', cutoff = 18)
    LowerMembrane_leaflet0 = L_low.groups(0)  ### upper
    LowerMembrane_leaflet1 = L_low.groups(1)  ### lower
    
    print(L_up, L_low)
    
    ######## Get number of positive and negative leaflets facing the Middle waters
    num_neg_lipids = len(UpperMembrane_leaflet1.select_atoms(f'resname {" ".join(neg_lipids.keys())}').resids) + len(LowerMembrane_leaflet0.select_atoms(f'resname {" ".join(neg_lipids.keys())}').resids)
    num_pos_lipids = len(UpperMembrane_leaflet1.select_atoms(f'resname {" ".join(pos_lipids.keys())}').resids) + len(LowerMembrane_leaflet0.select_atoms(f'resname {" ".join(pos_lipids.keys())}').resids)
    
    lipid_sum = num_pos_lipids*sum(pos_lipids.values()) + num_neg_lipids*sum(neg_lipids.values())
                        
    print(lipid_sum)
    
    
    if lipid_sum > 0:
        #### Add CL- ions in middle compartment
        tpr = 'ions_2.tpr'
        out = 'mem_solvent_ions.gro'
        top = 'system_W.top'
        cl_fn = str(np.abs(lipid_sum))
        pname_fn = 'CL'
        index_fn = 'index.ndx'
        !touch empty.mdp

        subprocess.call('gmx grompp -f empty.mdp -c stacked_membrane_renum_W_order.gro \
        -r stacked_membrane_renum_W_order.gro -p system_W.top -o ions_2.tpr -renum -maxwarn 2', shell=True)

        p = subprocess.Popen(['gmx', 'genion', '-s', tpr, '-o', out, '-p', top, '-nn', cl_fn, '-nname', pname_fn, '-n', index_fn], stdin=subprocess.PIPE)
        p.communicate(b'MiddleWater\n')
        p.wait()
    else:
        #### Add NA+ ions in middle compartment
        tpr = 'ions_2.tpr'
        out = 'mem_solvent_ions.gro'
        top = 'system_W.top'
        np_fn = str(np.abs(lipid_sum))
        pname_fn = 'NA'
        index_fn = 'index.ndx'
        !touch empty.mdp

        subprocess.call('gmx grompp -f empty.mdp -c stacked_membrane_renum_W_order.gro -r stacked_membrane_renum_W_order.gro -p system_W.top -o ions_2.tpr -renum -maxwarn 1', shell=True)

        p = subprocess.Popen(['gmx', 'genion', '-s', tpr, '-o', out, '-p', top, '-np', np_fn, '-pname', pname_fn, '-n', index_fn], stdin=subprocess.PIPE)
        p.communicate(b'MiddleWater\n')
        p.wait()
    
                
    #### Add ions in outside compartment
    tpr = 'ions.tpr'
    out = 'mem_solvent_ions.gro'
    top = 'system_W.top'
    index_fn = 'index.ndx'
    !touch empty.mdp

    subprocess.call('gmx grompp -f empty.mdp -c mem_solvent_ions.gro -r mem_solvent_ions.gro -p system_W.top\
    -o ions.tpr -renum -maxwarn 1', shell=True)

    p = subprocess.Popen(['gmx', 'genion', '-s', tpr, '-o', out, '-p', top, '-neutral', '-n', index_fn], stdin=subprocess.PIPE)
    p.communicate(b'OuterWater\n')
    p.wait()
    
    return

In [8]:
def solvate(not_tails):
    ############## Solvate system
    subprocess.call(f'gmx solvate -cp stacked_membrane_renum.gro -cs {water_loc}/water.gro -p system.top -o stacked_membrane_renum_W.gro -radius 0.21', shell=True)
    
    ##########################################
    ############## Assign Middle and Outter waters
    ##########################################
    u = md.Universe('stacked_membrane_renum_W.gro')
    #### Adapt topology file
    with open('system.top') as topinput:
        top = topinput.readlines()

    ### Select middle waters
    box_z = u.dimensions[2]
    top_leaflet = u.select_atoms('all and not name W and prop z > {}'.format(box_z/2))
    bot_leaflet = u.select_atoms('all and not name W and prop z < {}'.format(box_z/2))
    mem_top = top_leaflet.center_of_geometry()[2]-10
    mem_bot = bot_leaflet.center_of_geometry()[2]+10

    top[-1] = 'MiddleWater       {}\n'.format(len(u.select_atoms(f'name W and (prop z > {mem_bot} and prop z <= {mem_top})')))
    top.append('OuterWater       {}\n'.format(len(u.select_atoms(f'name W and (prop z > {mem_top} or prop z <= {mem_bot})'))))

    #### write the files
    with open("system_W.top", "w") as f:
        top = "".join(top)
        f.write(top)


    #### Re-order .gro file
    membrane = u.select_atoms('not name W')
    WNmiddle = u.select_atoms(f'name W and prop z <= {mem_top} and prop z > {mem_bot}')
    WNouter = u.select_atoms(f'name W and (prop z > {mem_top} or prop z <= {mem_bot})')

    outgoing = membrane + WNmiddle + WNouter
    outgoing.atoms.write('stacked_membrane_renum_W_order.gro')
    
       
    print (len(u.select_atoms(f'name W and (prop z > {mem_bot} and prop z < {mem_top})'))/330)
                              
    
    ##########################################
    ############## Create new index file
    ##########################################                          
    gro = 'stacked_membrane_renum_W_order.gro'
    u = md.Universe(gro)

    UpperMembrane = u.select_atoms(f'prop z > {box_z/2} and not name W NA CL')
    LowerMembrane = u.select_atoms(f'prop z < {box_z/2} and not name W NA CL')
    MiddleWater = u.select_atoms(f'name W NA and prop z < {mem_top} and prop z > {mem_bot}')
    OuterWater = u.select_atoms(f'name W NA and (prop z > {mem_top} or prop z < {mem_bot})')
    tails = u.select_atoms(f'not name {not_tails} W NA CL and not resname CHOL')
    water = u.select_atoms('name W')
    
    ##### Generate gro file for each membrane
    UpperMembrane.write('UpperMembrane.gro')
    LowerMembrane.write('LowerMembrane.gro')
    
    
    ##### Write new .ndx files
    LowerMembrane.write("index.ndx", mode="w", name= 'LowerMembrane')
    UpperMembrane.write("index.ndx", mode="a", name= 'UpperMembrane')
    MiddleWater.write("index.ndx", mode="a", name= 'MiddleWater')
    OuterWater.write("index.ndx", mode="a", name= 'OuterWater')
    tails.write("index.ndx", mode="a", name= 'tails')
    water.write("index.ndx", mode="a", name= 'water')

    u.atoms.write("index.ndx", mode="a", name= 'System')
    
    ############# Create flat bottom restraints for the water molecules
    #### Find COM of both leaflets
    LowerMembrane_com = LowerMembrane.center_of_geometry()/10
    UpperMembrane_com = UpperMembrane.center_of_geometry()/10
    
    print(LowerMembrane_com[2], UpperMembrane_com[2])

    zWNmiddle = (UpperMembrane_com[2]-LowerMembrane_com[2])/2-0.5 #atractive
    zWNouter = (UpperMembrane_com[2]-LowerMembrane_com[2])/2+0.5 #repulsive

    #### Adapt .itp file
    with open(f'{ff_loc}/WN_fbposres.itp') as topinput:
        top = topinput.readlines()

    top[7] = '[ position_restraints ]\n'
    top[8] = f'  1     2     5     {zWNmiddle}  50  ; attractive Z-layer\n'

    top[18] = '[ position_restraints ]\n'
    top[19] = f'  1     2     5     -{zWNouter}   50  ; repulsive Z-layer\n'

    #### write the files
    with open(f'{ff_loc}/WN_fbposres.itp', "w") as f:
        top = "".join(top)
        f.write(top)
        
    ###Create restraint reference at the center of the membrane
    tm = MiddleWater.positions
    tm[:,2]=np.full(MiddleWater.positions[:,2].shape, u.dimensions[2]/2)
    MiddleWater.positions = tm

    to = OuterWater.positions
    to[:,2]=np.full(OuterWater.positions[:,2].shape, u.dimensions[2]/2)
    OuterWater.positions = to

    u.atoms.write('restraint_reference.gro')
                                 
    return print('All done')

### Build system with stacked membranes
===============================================================================================

In [17]:
hydration = '22' #(~5 waters per lipid)
comp1 = f'-u POPC:640 -u PAPS:264 -u PAPC:56 -u POPE:264 -u DIPE:160 -u DPSM:640 -u CHOL:826 \
-l POPC:373 -l PAPC:213 -l POPE:133 -l DIPE:426 -l DPSM:293 -l CHOL:746 -l PAPS:666 -l SAP6:53 -x 10 -y 10 -z 8'

comp2 = f'-l MC3H:50 -l DPPC:10 -l CHOL:40 -x 10 -y 10 -z 8' 

forcefields = f'#include "{ff_loc}/martini_v3.0.0.itp"\n\
#include "{ff_loc}/Martini_v1_tails-2.itp"\n\
#include "{ff_loc}/martini_v3.0.0_ions_v1.itp"\n\
#include "{ff_loc}/martini_v3.0.0_solvents_v1.itp"\n\
#include "{ff_loc}/martini_v3.0_phosphoinositides_v1.0.itp"\n\
#include "{ff_loc}/martini_v3.0.0_phospholipids_v1.itp"\n\
#include "{ff_loc}/martini_v3.0_CHOL_dev20.itp"\n\
#include "{ff_loc}/WN_fbposres.itp"\n\
#include "{ff_loc}/MC3_MC3H.itp"\n'

### Run
build_system(comp1, comp2, hydration, forcefields, insane_loc)

; X: 10.000 (13 bins) Y: 10.000 (13 bins) in upper leaflet
; X: 10.000 (13 bins) Y: 10.000 (13 bins) in lower leaflet
; 169 lipids in upper leaflet, 169 lipids in lower leaflet
; NDX Solute 1 0
; Charge of protein: 0.000000
; NDX Membrane 1 3707
; Charge of membrane: -65.000000
; Total charge: -65.000000
; NDX Solvent 3708 0
; NDX System 1 3707
; "I mean, the good stuff is just INSANE" --Julia Ormond
; X: 10.000 (13 bins) Y: 10.000 (13 bins) in upper leaflet
; X: 10.000 (13 bins) Y: 10.000 (13 bins) in lower leaflet
; 169 lipids in upper leaflet, 169 lipids in lower leaflet
; NDX Solute 1 0
; Charge of protein: 0.000000
; NDX Membrane 1 3606
; Charge of membrane: 0.000000
; Total charge: 0.000000
; NDX Solvent 3607 0
; NDX System 1 3606
; "I mean, the good stuff is just INSANE" --Julia Ormond
                     :-) GROMACS - gmx editconf, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 3707 atoms
Volume: 800 nm^3, corresponds to roughly 360000 electrons
No velocities found

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. Bondi
van der Waals Volumes and Radii
J. Phys. Chem. 68 (1964) pp. 441-451
-------- -------- --- Thank You --- -------- --------

Adding line for

                     :-) GROMACS - gmx solvate, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working dir:  /local/mvalerio/LNP_Sanofi/5_Endossomal_fusion/PMFs/Scripts
Command line:
  gmx solvate -cp comp1_mem.gro -cs /DAMM/hosts/damm085/data1/mvalerio/Lisbeth/PMFs/common_files/water.gro -p topol_comp1_W.top -o comp1_mem_W.gro -radius 0.21

Reading solute configuration
Reading solvent configuration

Initialising inter-atomic distances...
Generating solvent configuration
Will generate new solvent configuration of 3x3x3 boxes
Solvent box contains 8009 atoms in 8009 residues
Removed 1783 solvent atoms due to solvent-solvent overlap
Removed 2633 solvent atoms due to solute-solvent overlap
Sorting configuration
Found 1 molecule type:
      W (   1 atoms):  3593 residues
Generated solvent containing 3593 atoms in 3593 residues
Writing generated configuration to comp1_mem_W.gro

Output configuration 

Setting the LD random seed to 1834844108

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'POPC'

Excluding 1 bonded neighbours molecule type 'PAPS'

Excluding 1 bonded neighbours molecule type 'PAPC'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'DIPE'

Excluding 1 bonded neighbours molecule type 'DPSM'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'POPC'

Excluding 1 bonded neighbours molecule type 'PAPC'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'DIPE'

Excluding 1 bonded neighbours molecule type 'DPSM'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'PAPS'

Excluding 1 bonded neighbours molecule type 'SAP6'

Excluding 1 bonded neighbours molecule type 'W'

Cleaning up constraints and constant bonded interactions with virt

Number of degrees of freedom in T-Coupling group rest is 20523.00
The integrator does not provide a ensemble temperature, there is no system ensemble temperature

GROMACS reminds you: "If Java had true garbage collection, most programs would delete themselves upon execution." (Robert Sewell)

                      :-) GROMACS - gmx mdrun, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working dir:  /local/mvalerio/LNP_Sanofi/5_Endossomal_fusion/PMFs/Scripts
Command line:
  gmx mdrun -v -deffnm em_comp1 -nt 10 -pin on -pinstride 1 -pinoffset 30



...

Setting the LD random seed to 1176353711

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'MC3H'

Excluding 1 bonded neighbours molecule type 'DPPC'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'MC3H'

Excluding 1 bonded neighbours molecule type 'DPPC'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'W'

Excluding 1 bonded neighbours molecule type 'CL'

Cleaning up constraints and constant bonded interactions with virtual sites

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 310 K

Calculated rlist for 1x1 atom pair-list as 1.306 nm, buffer size 0.206 nm

Set rlist, assuming 4x4 atom pair-list, to 1.215 nm, buffer size 0.115 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

This run will generate roughly 8 Mb of data


Reading file eq1_comp2_in.tpr, VERSION 2023.3 (single precision)
Changing nstlist from 20 to 25, rlist from 1.215 to 1.269

Update groups can not be used for this system because an incompatible virtual site type is used

1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 10 OpenMP threads 

Applying core pinning offset 30
starting mdrun 'INSANE! Membrane UpperLeaflet>MC3H:DPPC:CHOL=50.0:10.0:40.0 LowerLeaflet>MC3H:DPPC:CHOL=50.0:10.0:40.0 in water'
1000000 steps,  20000.0 ps.

Step 0  Warning: pressure scaling more than 1%, mu: 0.984893 0.984893 0.99067
step 0
Step 75  Warning: pressure scaling more than 1%, mu: 1.00475 1.00475 1.01206

Step 100  Warning: pressure scaling more than 1%, mu: 1.00444 1.00444 1.01188
step 999900, remaining wall clock time:     0 s          
Writing final coo

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 7105 atoms
Volume: 703.345 nm^3, corresponds to roughly 316500 electrons
Velocities found
    system size : 11.887 12.947  8.410 (nm)
    center      :  4.558  4.592  4.186 (nm)
    box vectors :  9.144  9.144  8.412 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  : 703.34               (nm^3)
    shift       :  0.442  0.408  6.814 (nm)
new center      :  5.000  5.000 11.000 (nm)
new box vectors : 10.000 10.000 22.000 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  :2200.00               (nm^3)


/DAMM/hosts/damm085/data1/mvalerio/my_env/lib/python3.12/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/DAMM/hosts/damm085/data1/mvalerio/my_env/lib/python3.12/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: T
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/DAMM/hosts/damm085/data1/mvalerio/my_env/lib/python3.12/site-packages/MDAnalysis/coordinates/GRO.py:479: UserWarning: missing dimension - setting unit cell to zeroed box [0., 0., 0.]
  warnings.warn(wmsg)
                     :-) GROMACS - gmx editconf, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working dir:  /local/mvalerio/LNP_Sanofi/5_Endossomal_fusio

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 7313 atoms
Volume: 0 nm^3, corresponds to roughly 0 electrons
No velocities found
    system size : 11.887 13.072 12.390 (nm)
    center      :  4.936  4.842  7.708 (nm)
    box vectors :  0.000  0.000  0.000 (nm)
    box angles  :   0.00   0.00   0.00 (degrees)
    box volume  :   0.00               (nm^3)
    shift       :  0.064  0.158  1.292 (nm)
new center      :  5.000  5.000  9.000 (nm)
new box vectors : 10.000 10.000 18.000 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  :1800.00               (nm^3)
rm: cannot remove '*#': No such file or directory


In [19]:
## Calculate the inconsistency in terms of membrane size to scale the bilayers
scaling_factor = area_per_lipid('eq1_comp2_in.gro', 'eq1_comp2_in.xtc')[0]/area_per_lipid('eq1_comp1.gro', 'eq1_comp1.xtc')[0]
scaling_factor

1.1155922794412363

In [20]:
#### Obtain new box size
scaled_x = 10 + scaling_factor
scaled_y = 10 + scaling_factor

#### Re-run with scaling factor
hydration = '22' #(~5 waters per lipid)
comp1 = f'-u POPC:640 -u PAPS:264 -u PAPC:56 -u POPE:264 -u DIPE:160 -u DPSM:640 -u CHOL:826 \
-l POPC:373 -l PAPC:213 -l POPE:133 -l DIPE:426 -l DPSM:293 -l CHOL:746 -l PAPS:666 -l SAP6:53 -x 10 -y 10 -z 8'

comp2 = f'-l MC3H:50 -l DPPC:10 -l CHOL:40 -x {scaled_x} -y {scaled_y} -z 8' 

forcefields = f'#include "{ff_loc}/martini_v3.0.0.itp"\n\
#include "{ff_loc}/Martini_v1_tails-2.itp"\n\
#include "{ff_loc}/martini_v3.0.0_ions_v1.itp"\n\
#include "{ff_loc}/martini_v3.0.0_solvents_v1.itp"\n\
#include "{ff_loc}/martini_v3.0_phosphoinositides_v1.0.itp"\n\
#include "{ff_loc}/martini_v3.0.0_phospholipids_v1.itp"\n\
#include "{ff_loc}/martini_v3.0_CHOL_dev20.itp"\n\
#include "{ff_loc}/WN_fbposres.itp"\n\
#include "{ff_loc}/MC3_MC3H.itp"\n'

### Run
build_system(comp1, comp2, hydration, forcefields, insane_loc)

; X: 10.000 (13 bins) Y: 10.000 (13 bins) in upper leaflet
; X: 10.000 (13 bins) Y: 10.000 (13 bins) in lower leaflet
; 169 lipids in upper leaflet, 169 lipids in lower leaflet
; NDX Solute 1 0
; Charge of protein: 0.000000
; NDX Membrane 1 3707
; Charge of membrane: -65.000000
; Total charge: -65.000000
; NDX Solvent 3708 0
; NDX System 1 3707
; "I mean, the good stuff is just INSANE" --Julia Ormond
; X: 11.116 (14 bins) Y: 11.116 (14 bins) in upper leaflet
; X: 11.116 (14 bins) Y: 11.116 (14 bins) in lower leaflet
; 196 lipids in upper leaflet, 196 lipids in lower leaflet
; NDX Solute 1 0
; Charge of protein: 0.000000
; NDX Membrane 1 4212
; Charge of membrane: 0.000000
; Total charge: 0.000000
; NDX Solvent 4213 0
; NDX System 1 4212
; "I mean, the good stuff is just INSANE" --Julia Ormond
                     :-) GROMACS - gmx editconf, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 3707 atoms
Volume: 800 nm^3, corresponds to roughly 360000 electrons
No velocities found

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. Bondi
van der Waals Volumes and Radii
J. Phys. Chem. 68 (1964) pp. 441-451
-------- -------- --- Thank You --- -------- --------

Adding line for

Removed 1783 solvent atoms due to solvent-solvent overlap
Removed 2607 solvent atoms due to solute-solvent overlap
Sorting configuration
Found 1 molecule type:
      W (   1 atoms):  3619 residues
Generated solvent containing 3619 atoms in 3619 residues
Writing generated configuration to comp1_mem_W.gro

Back Off! I just backed up comp1_mem_W.gro to ./#comp1_mem_W.gro.1#

Output configuration contains 7326 atoms in 3948 residues
Volume                 :         800 (nm^3)
Density                :     1484.14 (g/l)
Number of solvent molecules:   3619   

Processing topology

Back Off! I just backed up topol_comp1_W.top to ./#topol_comp1_W.top.1#

GROMACS reminds you: "It seemed a good idea at first" (Gerrit Groenhof)

                      :-) GROMACS - gmx grompp, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working dir:  /local/mvalerio/LNP_Sanofi/5_Endossomal_fusion/PMFs/Scripts
Command lin

Setting the LD random seed to -21020681

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'POPC'

Excluding 1 bonded neighbours molecule type 'PAPS'

Excluding 1 bonded neighbours molecule type 'PAPC'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'DIPE'

Excluding 1 bonded neighbours molecule type 'DPSM'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'POPC'

Excluding 1 bonded neighbours molecule type 'PAPC'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'DIPE'

Excluding 1 bonded neighbours molecule type 'DPSM'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'PAPS'

Excluding 1 bonded neighbours molecule type 'SAP6'

Excluding 1 bonded neighbours molecule type 'W'

Cleaning up constraints and constant bonded interactions with virtu

Number of degrees of freedom in T-Coupling group rest is 20601.00
The integrator does not provide a ensemble temperature, there is no system ensemble temperature

Back Off! I just backed up em_comp1.tpr to ./#em_comp1.tpr.1#

GROMACS reminds you: "A tidy laboratory means a lazy chemist." (J.J. Berzelius)

                      :-) GROMACS - gmx mdrun, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working dir:  /local/mvalerio/LNP_Sanofi/5_Endossomal_fusion/PMFs/Scripts
Command line:
  gmx mdrun -v -deffnm em_comp1 -nt 10 -pin on -pinstride 1 -pinoffset 30



...

Setting the LD random seed to -33820181

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'MC3H'

Excluding 1 bonded neighbours molecule type 'DPPC'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'MC3H'

Excluding 1 bonded neighbours molecule type 'DPPC'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'W'

Excluding 1 bonded neighbours molecule type 'CL'

Cleaning up constraints and constant bonded interactions with virtual sites

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 310 K

Calculated rlist for 1x1 atom pair-list as 1.305 nm, buffer size 0.205 nm

Set rlist, assuming 4x4 atom pair-list, to 1.215 nm, buffer size 0.115 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

This run will generate roughly 9 Mb of data



Back Off! I just backed up eq1_comp2_in.log to ./#eq1_comp2_in.log.1#
Reading file eq1_comp2_in.tpr, VERSION 2023.3 (single precision)
Changing nstlist from 20 to 25, rlist from 1.215 to 1.269

Update groups can not be used for this system because an incompatible virtual site type is used

1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 10 OpenMP threads 

Applying core pinning offset 30

Back Off! I just backed up eq1_comp2_in.xtc to ./#eq1_comp2_in.xtc.1#

Back Off! I just backed up eq1_comp2_in.edr to ./#eq1_comp2_in.edr.1#
starting mdrun 'INSANE! Membrane UpperLeaflet>MC3H:DPPC:CHOL=50.0:10.0:40.0 LowerLeaflet>MC3H:DPPC:CHOL=50.0:10.0:40.0 in water'
1000000 steps,  20000.0 ps.

Step 0  Warning: pressure scaling more than 1%, mu: 0.985446 0.985446 0.990056
step 0
Step 100  Warning

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 8600 atoms
Volume: 863.52 nm^3, corresponds to roughly 388500 electrons
Velocities found
    system size : 12.524 12.061  9.254 (nm)
    center      :  4.762  4.855  4.620 (nm)
    box vectors :  9.655  9.655  9.264 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  : 863.52               (nm^3)
    shift       :  0.238  0.145  6.380 (nm)
new center      :  5.000  5.000 11.000 (nm)
new box vectors : 10.000 10.000 22.000 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  :2200.00               (nm^3)


/DAMM/hosts/damm085/data1/mvalerio/my_env/lib/python3.12/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/DAMM/hosts/damm085/data1/mvalerio/my_env/lib/python3.12/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: T
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/DAMM/hosts/damm085/data1/mvalerio/my_env/lib/python3.12/site-packages/MDAnalysis/coordinates/GRO.py:479: UserWarning: missing dimension - setting unit cell to zeroed box [0., 0., 0.]
  warnings.warn(wmsg)
                     :-) GROMACS - gmx editconf, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working dir:  /local/mvalerio/LNP_Sanofi/5_Endossomal_fusio

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 7919 atoms
Volume: 0 nm^3, corresponds to roughly 0 electrons
No velocities found
    system size : 12.650 12.061 12.980 (nm)
    center      :  4.794  4.929  7.912 (nm)
    box vectors :  0.000  0.000  0.000 (nm)
    box angles  :   0.00   0.00   0.00 (degrees)
    box volume  :   0.00               (nm^3)
    shift       :  0.206  0.071  1.088 (nm)
new center      :  5.000  5.000  9.000 (nm)
new box vectors : 10.000 10.000 18.000 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  :1800.00               (nm^3)


In [23]:
## Check the box sizes match 
!tail eq1_comp1.gro
!tail eq1_comp2_in.gro

 3940ION     NA 7318   0.581   3.344   2.598 -0.2184 -0.0278 -0.0602
 3941ION     NA 7319   1.649   1.259   1.983  0.0654  0.1726  0.3719
 3942ION     NA 7320   2.528   7.726   7.846  0.0739 -0.2665  0.3699
 3943ION     NA 7321   3.723   5.878   2.155  0.1507  0.3152  0.2357
 3944ION     NA 7322   6.315   2.893   1.507  0.0639 -0.2550  0.1729
 3945ION     NA 7323   4.270   3.482   6.193  0.2119  0.1128 -0.1714
 3946ION     NA 7324   3.240   4.967   1.583  0.0397  0.1190  0.0400
 3947ION     NA 7325   7.694   6.461   7.017 -0.0207  0.0629  0.0560
 3948ION     NA 7326   6.919   9.101   1.758  0.0152 -0.1243  0.0954
   9.57512   9.57512   8.56850
 4770ION     CL 8592   4.030   5.354   1.962 -0.2684 -0.4282 -0.0721
 4771ION     CL 8593   7.078   7.800   2.061  0.4847  0.1125 -0.1771
 4772ION     CL 8594   9.200   3.273   2.280  0.4055  0.2888 -0.1975
 4773ION     CL 8595   0.645   8.326   7.491  0.0976  0.3912 -0.1172
 4774ION     CL 8596   0.092   6.035   1.672  0.0609  0.2755  0.3367
 47

## Create topology
===============================================================================================

In [9]:
############# Create topology file
with open('topol_comp1_W.top') as topinput:
    top = topinput.readlines()[:-2]

## write the file
with open("system.top", "w") as f:
    top = "".join(top)
    f.write(top)
    
############# Add Comp2 membrane lipids
with open('system.top') as topinput:
    sys = topinput.read()
    
## Find line were molecules start    
lookup = '[ molecules ]'    
with open('topol_comp2.top') as topinput:
    for num, line in enumerate(topinput,1):
        if lookup in line:
            mem2 = topinput.read()[num:]
    
sys += ';membrane2'
sys += mem2

with open("system.top", "w") as f:
    f.write(sys)

In [10]:
!cat system.top

#include "/DAMM/hosts/damm085/data1/mvalerio/Lisbeth/PMFs/itp//martini_v3.0.0.itp"
#include "/DAMM/hosts/damm085/data1/mvalerio/Lisbeth/PMFs/itp//Martini_v1_tails-2.itp"
#include "/DAMM/hosts/damm085/data1/mvalerio/Lisbeth/PMFs/itp//martini_v3.0.0_ions_v1.itp"
#include "/DAMM/hosts/damm085/data1/mvalerio/Lisbeth/PMFs/itp//martini_v3.0.0_solvents_v1.itp"
#include "/DAMM/hosts/damm085/data1/mvalerio/Lisbeth/PMFs/itp//martini_v3.0_phosphoinositides_v1.0.itp"
#include "/DAMM/hosts/damm085/data1/mvalerio/Lisbeth/PMFs/itp//martini_v3.0.0_phospholipids_v1.itp"
#include "/DAMM/hosts/damm085/data1/mvalerio/Lisbeth/PMFs/itp//martini_v3.0_CHOL_dev20.itp"
#include "/DAMM/hosts/damm085/data1/mvalerio/Lisbeth/PMFs/itp//WN_fbposres.itp"
#include "/DAMM/hosts/damm085/data1/mvalerio/Lisbeth/PMFs/itp//MC3_MC3H.itp"

[ system ]
; name
INSANE! Membrane UpperLeaflet>POPC:PAPS:PAPC:POPE:DIPE:DPSM:CHOL=640.0:264.0:56.0:264.0:160.0:640.0:826.0 LowerLeaflet>POPC:PAPC:POPE:DIPE:DPSM:CHOL:PAPS:SAP6=373.0:213.0:1

### Solvate and equilibrate system
===============================================================================================

In [11]:
not_tails = 'NP CN GLA CX NC3 NH3 CNO C1 C2 C3 C4 P4 P5 PO4 GL1 GL2 AM1 AM2 ROH BB SC1 SC2 SC3 SC4 SC5 CA' #### atoms belonging to the 'heads' of the lipids
neg_lipids = {'PAPS SAP6': -1}  ##### write the name of the lipid and the charge without + or -
pos_lipids = {f'{IL_name}': +1}

### Run
solvate(not_tails)
neutralize(neg_lipids, pos_lipids)

                     :-) GROMACS - gmx solvate, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working dir:  /local/mvalerio/LNP_Sanofi/5_Endossomal_fusion/PMFs/Scripts
Command line:
  gmx solvate -cp stacked_membrane_renum.gro -cs /DAMM/hosts/damm085/data1/mvalerio/Lisbeth/PMFs/common_files/water.gro -p system.top -o stacked_membrane_renum_W.gro -radius 0.21

Reading solute configuration
Reading solvent configuration

Initialising inter-atomic distances...
Generating solvent configuration
Will generate new solvent configuration of 3x3x5 boxes
Solvent box contains 17035 atoms in 17035 residues
Removed 2792 solvent atoms due to solvent-solvent overlap
Removed 6392 solvent atoms due to solute-solvent overlap
Sorting configuration
Found 1 molecule type:
      W (   1 atoms):  7851 residues
Generated solvent containing 7851 atoms in 7851 residues
Writing generated configuration to stacked_membrane_


         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. Bondi
van der Waals Volumes and Radii
J. Phys. Chem. 68 (1964) pp. 441-451
-------- -------- --- Thank You --- -------- --------

Adding line for 7851 solvent molecules with resname (W) to topology file (system.top)


/DAMM/hosts/damm085/data1/mvalerio/my_env/lib/python3.12/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/DAMM/hosts/damm085/data1/mvalerio/my_env/lib/python3.12/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: D
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/DAMM/hosts/damm085/data1/mvalerio/my_env/lib/python3.12/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: G
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/DAMM/hosts/damm085/data1/mvalerio/my_env/lib/python3.12/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: R
  w

5.618181818181818
5.530322004984884 12.05552527962827
All done


/DAMM/hosts/damm085/data1/mvalerio/my_env/lib/python3.12/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/DAMM/hosts/damm085/data1/mvalerio/my_env/lib/python3.12/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: D
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/DAMM/hosts/damm085/data1/mvalerio/my_env/lib/python3.12/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: G
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/DAMM/hosts/damm085/data1/mvalerio/my_env/lib/python3.12/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: R
  w

<LeafletFinder('name PO4 NP', cutoff=20.0 A) with 234 atoms in 5 groups> <LeafletFinder('name PO4 NP', cutoff=18.0 A) with 238 atoms in 2 groups>
-14


                      :-) GROMACS - gmx grompp, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working dir:  /local/mvalerio/LNP_Sanofi/5_Endossomal_fusion/PMFs/Scripts
Command line:
  gmx grompp -f empty.mdp -c stacked_membrane_renum_W_order.gro -r stacked_membrane_renum_W_order.gro -p system_W.top -o ions_2.tpr -renum -maxwarn 1


NOTE 1 [file empty.mdp]:
  For a correct single-point energy evaluation with nsteps = 0, use
  continuation = yes to avoid constraining the input coordinates.


NOTE 2 [file system_W.top, line 40]:
  System has non-zero total charge: 131.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.



atom name 7920 in system_W.top and stacked_membrane_renum_W_order.gro does not match (WN - W)

atom name 7921 in system_W.top and stacked_membrane_renu

Setting the LD random seed to 1052766199

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'POPC'

Excluding 1 bonded neighbours molecule type 'PAPS'

Excluding 1 bonded neighbours molecule type 'PAPC'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'DIPE'

Excluding 1 bonded neighbours molecule type 'DPSM'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'POPC'

Excluding 1 bonded neighbours molecule type 'PAPC'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'DIPE'

Excluding 1 bonded neighbours molecule type 'DPSM'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'PAPS'

Excluding 1 bonded neighbours molecule type 'SAP6'

Excluding 1 bonded neighbours molecule type 'MC3H'

Excluding 1 bonded neighbours molecule type 'DPPC'

Excluding 1

                      :-) GROMACS - gmx grompp, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working dir:  /local/mvalerio/LNP_Sanofi/5_Endossomal_fusion/PMFs/Scripts
Command line:
  gmx grompp -f empty.mdp -c mem_solvent_ions.gro -r mem_solvent_ions.gro -p system_W.top -o ions.tpr -renum -maxwarn 1


NOTE 1 [file empty.mdp]:
  For a correct single-point energy evaluation with nsteps = 0, use
  continuation = yes to avoid constraining the input coordinates.


NOTE 2 [file system_W.top, line 41]:
  System has non-zero total charge: 145.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.




NOTE 3 [file system_W.top, line 41]:
  For energy conservation with LINCS, lincs_iter should be 2 or larger.


Number of degrees of freedom in T-Coupling group rest is 42843.00
The

Setting the LD random seed to -742426642

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'POPC'

Excluding 1 bonded neighbours molecule type 'PAPS'

Excluding 1 bonded neighbours molecule type 'PAPC'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'DIPE'

Excluding 1 bonded neighbours molecule type 'DPSM'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'POPC'

Excluding 1 bonded neighbours molecule type 'PAPC'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'DIPE'

Excluding 1 bonded neighbours molecule type 'DPSM'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'PAPS'

Excluding 1 bonded neighbours molecule type 'SAP6'

Excluding 1 bonded neighbours molecule type 'MC3H'

Excluding 1 bonded neighbours molecule type 'DPPC'

Excluding 1

### Minimize and equilibrate system with cell and viral membranes
===============================================================================================

In [12]:
### Minimize and equilibrate the double bilayer system
minimize_equilibrate()

                      :-) GROMACS - gmx grompp, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working dir:  /local/mvalerio/LNP_Sanofi/5_Endossomal_fusion/PMFs/Scripts
Command line:
  gmx grompp -f /DAMM/hosts/damm085/data1/mvalerio/Lisbeth/PMFs/mdps/em_2.mdp -c mem_solvent_ions.gro -r restraint_reference.gro -p system_W.top -o em2.tpr

Ignoring obsolete mdp entry 'ns_type'
Number of degrees of freedom in T-Coupling group rest is 43593.00
The integrator does not provide a ensemble temperature, there is no system ensemble temperature

GROMACS reminds you: "I'll Match Your DNA" (Red Hot Chili Peppers)

                      :-) GROMACS - gmx mdrun, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working dir:  /local/mvalerio/LNP_Sanofi/5_Endossomal_fusion/PMFs/Scripts
Command line:
  gmx mdrun -v -deffnm 

Setting the LD random seed to -755175562

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'POPC'

Excluding 1 bonded neighbours molecule type 'PAPS'

Excluding 1 bonded neighbours molecule type 'PAPC'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'DIPE'

Excluding 1 bonded neighbours molecule type 'DPSM'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'POPC'

Excluding 1 bonded neighbours molecule type 'PAPC'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'DIPE'

Excluding 1 bonded neighbours molecule type 'DPSM'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'PAPS'

Excluding 1 bonded neighbours molecule type 'SAP6'

Excluding 1 bonded neighbours molecule type 'MC3H'

Excluding 1 bonded neighbours molecule type 'DPPC'

Excluding 1

Reading file em2.tpr, VERSION 2023.3 (single precision)
Update groups can not be used for this system because an incompatible virtual site type is used

1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 10 OpenMP threads 

Applying core pinning offset 30

Steepest Descents:
   Tolerance (Fmax)   =  1.00000e+01
   Number of steps    =        10000
Step=    0, Dmax= 1.0e-02 nm, Epot=  2.46859e+08 Fmax= 9.53865e+09, atom= 2292
Step=    1, Dmax= 1.0e-02 nm, Epot=  1.22168e+08 Fmax= 1.69977e+09, atom= 2292
Step=    2, Dmax= 1.2e-02 nm, Epot=  3.94911e+07 Fmax= 3.43456e+08, atom= 2292
Step=    3, Dmax= 1.4e-02 nm, Epot=  1.52630e+07 Fmax= 5.99876e+07, atom= 5931
Step=    4, Dmax= 1.7e-02 nm, Epot=  4.41417e+06 Fmax= 1.17828e+07, atom= 6785
Step=    5, Dmax= 2.1e-02 nm, Epot=  1.50216e+06 Fmax

Setting the LD random seed to -672208173

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'POPC'

Excluding 1 bonded neighbours molecule type 'PAPS'

Excluding 1 bonded neighbours molecule type 'PAPC'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'DIPE'

Excluding 1 bonded neighbours molecule type 'DPSM'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'POPC'

Excluding 1 bonded neighbours molecule type 'PAPC'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'DIPE'

Excluding 1 bonded neighbours molecule type 'DPSM'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'PAPS'

Excluding 1 bonded neighbours molecule type 'SAP6'

Excluding 1 bonded neighbours molecule type 'MC3H'

Excluding 1 bonded neighbours molecule type 'DPPC'

Excluding 1

Reading file eq1.tpr, VERSION 2023.3 (single precision)
Changing nstlist from 20 to 25, rlist from 1.215 to 1.267

Update groups can not be used for this system because an incompatible virtual site type is used

1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 10 OpenMP threads 

Applying core pinning offset 30
starting mdrun 'INSANE! Membrane UpperLeaflet>POPC:PAPS:PAPC:POPE:DIPE:DPSM:CHOL=640.0:264.0:56.0:264.0:160.0:640.0:826.0 LowerLeaflet>POPC:PAPC:POPE:DIPE:DPSM:CHOL:PAPS:SAP6=373.0:213.0:133.0:426.0:293.0:746.0:666.0:53.0 in water'
1000000 steps,  20000.0 ps.

Step 0  Warning: pressure scaling more than 1%, mu: 0.983944 0.983944 0.987227
step 999900, remaining wall clock time:     0 s          
Writing final coordinates.
step 1000000, remaining wall clock time:     0 s          

rm: cannot remove '*#': No such file or directory


In [13]:
## Calculate the number of waters in the midle compartmente. Expecting a number between 3 and 6, if not change "hydration".
u = md.Universe('eq1.gro')
print(len(np.unique(u.select_atoms('name WN').resids))/333)

5.525525525525525


===============================================================================

In [14]:
!cat system_W.top

#include "/DAMM/hosts/damm085/data1/mvalerio/Lisbeth/PMFs/itp//martini_v3.0.0.itp"
#include "/DAMM/hosts/damm085/data1/mvalerio/Lisbeth/PMFs/itp//Martini_v1_tails-2.itp"
#include "/DAMM/hosts/damm085/data1/mvalerio/Lisbeth/PMFs/itp//martini_v3.0.0_ions_v1.itp"
#include "/DAMM/hosts/damm085/data1/mvalerio/Lisbeth/PMFs/itp//martini_v3.0.0_solvents_v1.itp"
#include "/DAMM/hosts/damm085/data1/mvalerio/Lisbeth/PMFs/itp//martini_v3.0_phosphoinositides_v1.0.itp"
#include "/DAMM/hosts/damm085/data1/mvalerio/Lisbeth/PMFs/itp//martini_v3.0.0_phospholipids_v1.itp"
#include "/DAMM/hosts/damm085/data1/mvalerio/Lisbeth/PMFs/itp//martini_v3.0_CHOL_dev20.itp"
#include "/DAMM/hosts/damm085/data1/mvalerio/Lisbeth/PMFs/itp//WN_fbposres.itp"
#include "/DAMM/hosts/damm085/data1/mvalerio/Lisbeth/PMFs/itp//MC3_MC3H.itp"

[ system ]
; name
INSANE! Membrane UpperLeaflet>POPC:PAPS:PAPC:POPE:DIPE:DPSM:CHOL=640.0:264.0:56.0:264.0:160.0:640.0:826.0 LowerLeaflet>POPC:PAPC:POPE:DIPE:DPSM:CHOL:PAPS:SAP6=373.0:213.0:1